In [1]:
!pip install networkx

In [11]:
import torch
import torch.nn.functional as F
import networkx as nx
import numpy as np
import torch.optim as optim
from itertools import combinations
import pickle

In [12]:
# Define a graph
n = 20  # number of nodes
m = 120  # number of edges
G = nx.gnm_random_graph(n, m, seed = 0)
complement_G = nx.complement(G)

# ### Visualize the graph
# nx.draw(G, with_labels=True, font_weight='bold')

# print(G.edges())


In [13]:
### Obtain the A_G matrix

adjacency_matrix = nx.adjacency_matrix(G)
adjacency_matrix_dense = adjacency_matrix.todense()
adjacency_matrix_tensor = torch.tensor(adjacency_matrix_dense, dtype=torch.float32)

### Obtain the A_G_hat matrix

adjacency_matrix_comp = nx.adjacency_matrix(complement_G)
adjacency_matrix_dense_comp = adjacency_matrix_comp.todense()
adjacency_matrix_tensor_comp = torch.tensor(adjacency_matrix_dense_comp, dtype=torch.float32)

### Define the loss

def loss_function(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, gamma, beta):
    ## with edges of the comp graph:
    loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X) - (beta/2) * (Matrix_X.T @ (adjacency_matrix_tensor_comp) @ Matrix_X)
    return loss


In [14]:
def is_independent_set(adjacency_matrix, mis_tensor):
    if len(mis_tensor) == 0 or len(mis_tensor) == 1:
      return False  # An empty set cannot be a maximal independent set. We are also igonorig checking the case where only one node is included in the set
    for node in mis_tensor:
        neighbors = adjacency_matrix[:,node].nonzero()
        if torch.any(torch.isin(neighbors, mis_tensor)):
            return False
    return True

def is_maximal_independent_set(adjacency_matrix, mis_tensor):
    if is_independent_set(adjacency_matrix, mis_tensor) is False:
      return False
    neighbors_set = torch.unique(torch.cat([adjacency_matrix[:,node].nonzero() for node in mis_tensor]))
    return neighbors_set.size()[0] + len(mis_tensor) == adjacency_matrix.size()[0]

In [15]:
# # Let MIS_tensor = {0,11,12,18}
# MIS_tensor_test = torch.tensor([0,11,12,18])
# print(is_maximal_independent_set(adjacency_matrix_tensor, MIS_tensor_test))

# #print(adjacency_matrix_tensor)

# #is_maximal_independent_set(adjacency_matrix_tensor, MIS_tensor)

In [ ]:
# Optimization loop:
# Initialization:
torch.manual_seed(115)
lower_bound = 0.01
upper_bound = 1
Matrix_X = (upper_bound - lower_bound) * torch.rand(n) + lower_bound
Matrix_X.requires_grad_(True)

# Optimization parameters
learning_rate_alpha = 0.1
number_of_iterations_T = 100000


# Define Optimizer over matrix X
optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)

Best_MIS_size = 0
for iteration_t in range(number_of_iterations_T):

    loss = loss_function(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, gamma = 500, beta = 1)

    # Gradient descent:
    optimizer.zero_grad()  # Clear gradients for the next iteration
    loss.backward()  # Backpropagation
    optimizer.step()  # Update the parameters

    # Box-constraining:
    Matrix_X.data[Matrix_X>=1] =1
    Matrix_X.data[Matrix_X<=0] =0

    MIS_list = []
    for node in range(n):
      if Matrix_X[node] > 0:
        MIS_list.append(node)
    MIS_tensor = torch.tensor(MIS_list)


    # Iteration logger every XX iterations:
    if (iteration_t + 1) % 100 == 0:
        print(f"Step {iteration_t + 1}/{number_of_iterations_T}, Loss: {loss.item()}, MIS_tensor: {MIS_tensor}")
    if is_maximal_independent_set(adjacency_matrix_tensor, MIS_tensor):
      if len(MIS_tensor) > Best_MIS_size: # This is to save our current best so far
        Best_MIS_size = len(MIS_tensor)

      print("+++++++++++ A MIS is found with size: ", [[[len(MIS_tensor)]]], "++++ BEST so far",[Best_MIS_size], "++++++++++++", MIS_tensor.squeeze())

      #### Restart:
      Matrix_X = (upper_bound - lower_bound) * torch.rand(n) + lower_bound
      Matrix_X.requires_grad_(True)
      optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)

Step 100/100000, Loss: -0.007432860787957907, MIS_tensor: tensor([ 5,  7, 13, 14, 17])
+++++++++++ A MIS is found with size:  [[[3]]] ++++ BEST so far [3] ++++++++++++ tensor([ 5,  7, 14])
Step 200/100000, Loss: -3.1025285807118053e-06, MIS_tensor: tensor([ 5,  6,  7,  8, 11, 13, 16, 18])
+++++++++++ A MIS is found with size:  [[[4]]] ++++ BEST so far [4] ++++++++++++ tensor([ 1,  7, 13, 16])
Step 300/100000, Loss: 0.0, MIS_tensor: tensor([])
+++++++++++ A MIS is found with size:  [[[4]]] ++++ BEST so far [4] ++++++++++++ tensor([ 7, 13, 16, 17])
Step 400/100000, Loss: 0.0, MIS_tensor: tensor([])
Step 500/100000, Loss: -0.049827784299850464, MIS_tensor: tensor([ 4,  6, 11, 14, 18])
+++++++++++ A MIS is found with size:  [[[4]]] ++++ BEST so far [4] ++++++++++++ tensor([ 4, 11, 14, 18])
Step 600/100000, Loss: -0.005315418355166912, MIS_tensor: tensor([ 3,  5,  7, 13, 14, 16, 17])
+++++++++++ A MIS is found with size:  [[[4]]] ++++ BEST so far [4] ++++++++++++ tensor([ 5,  7, 13, 17])
St